In [16]:
# importing libraries
from constants import *
import os
from dotenv import load_dotenv
import subprocess
import json
from eth_account import Account
from bit import PrivateKeyTestnet
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from bit.network import NetworkAPI

load_dotenv()

True

In [2]:
# calling mnemonic environment variable
mnemonic = os.getenv('MNEMONIC')

In [3]:
# function to derive wallets
def derive_wallets (mnemonic, coin, number):
    
    command = f"./derive -g --mnemonic='{mnemonic}' --coin='{coin}' --numderive='{number}' --cols=address,index,path,address,privkey,pubkey,pubkeyhash,xprv,xpub --format=json"
    process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = process.communicate()
    process_status = process.wait()
    keys = json.loads(output)
    
    return keys

In [4]:
# function to call derive_wallets function for BTCTEST and ETH currencies and output a dictionary
# with derived information for three accounts per coin
def coins ():

    coin_dict = {
        'btc-test' : derive_wallets(mnemonic, BTCTEST, 3),
        'eth' : derive_wallets(mnemonic, ETH, 3)
    }
    
    return coin_dict

In [5]:
# testing output
coins()

{'btc-test': [{'address': 'mhcPGasbdmB8gfc3bZQ3ry8dvPzYN46A2P',
   'index': 0,
   'path': "m/44'/1'/0'/0/0",
   'privkey': 'cTT72QfekYdz5kQp1gKHfqwPdAowF3YBSR5xfKaHibzGdLLbRCdN',
   'pubkey': '038d0c73ee96819b7a4ad135db8ba8b546a6e24ec7d6ff96a7f38554545b7aefdd',
   'pubkeyhash': '16f72ddec27cba3b276c460db3ed5263a9acdde9',
   'xprv': 'tprv8jjHJ5it1VbyoftVyaxNwmeiBr2UFZWkQvbmr7ixKJX98V85GpfsU5AreC9Thk3KYjAhq3deyuwSk59RisPkftugFqZe6Q2oN7cugGeeXum',
   'xpub': 'tpubDGRKSVm89sHeh8vHsEcyMBJpksYQQthezECZ8dmFjaKXxyNquDVTeZnipLn6dYCXUzVKQANCxviHek9HLK667GtBKqj8Y3NH6adb6izb7hg'},
  {'address': 'mxWxxZF5pxTePC1pzvD8uDDRfnjZ7DX6UQ',
   'index': 1,
   'path': "m/44'/1'/0'/0/1",
   'privkey': 'cPernSsfQYrp6qvYD4ixtsvrYT9FpEZiZR2QbiLS8bVizK89kyQx',
   'pubkey': '0398abfc88ce33e4f473796310f13d6c820ab72db5a2aff6aa412ac815ad923167',
   'pubkeyhash': 'ba7ab145b283301ab0be0a22520aeb23f64faa2a',
   'xprv': 'tprv8jjHJ5it1VbyqUVR68v5oaNPhcWzFTy3tUAM51fwVYM8RGrC7CHPpErzKT5nzi5TsWgHRh9USMbeB7uznTsGo6GJ1QD2ATXxt

In [6]:
# testing a call of a private key from the dictionary output
coins()[ETH][0]['privkey']

'0x923a3330959d0976c37fdcaf3e5d94012a81926fdbf27a72ceba9630e4d1adf4'

In [7]:
# function to convert private key into a readable format for web3 / bit
def priv_key_to_account (coin, priv_key):
    
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [8]:
# setting sender ETH account
eth_account = priv_key_to_account(ETH,coins()[ETH][0]['privkey'])

print(eth_account)

In [18]:
# setting a receiver ETH address
eth_address = coins()[ETH][1]['address']
print(address)

mxWxxZF5pxTePC1pzvD8uDDRfnjZ7DX6UQ


In [9]:
# setting sender BTCTEST account
btctest_account = priv_key_to_account(BTCTEST,coins()[BTCTEST][0]['privkey'])

print(btctest_account)

<PrivateKeyTestnet: mhcPGasbdmB8gfc3bZQ3ry8dvPzYN46A2P>


In [10]:
# setting a receiver BTCTEST address
btctest_address = coins()[BTCTEST][1]['address']
print(address)

mxWxxZF5pxTePC1pzvD8uDDRfnjZ7DX6UQ


In [11]:
# setting up Web3 config
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer =0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [12]:
# function to create raw, unsigned transaction
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas({
            "from": account.address, 
            "to": to, 
            "value": amount
        })
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gas": gasEstimate,
            "gasPrice": w3.eth.gasPrice,
            "nonce": w3.eth.getTransactionCount(account.address),
            "chainID": w3.eth.net.getId()
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [13]:
# testing transaction with BTC
create_tx(BTCTEST,btctest_account,btctest_address,0.001)

'{"unspents":[{"amount":788834,"confirmations":0,"script":"76a91416f72ddec27cba3b276c460db3ed5263a9acdde988ac","txid":"a9e40e1f500894aac4a2441f5b451bdd19c30769b7c361b5e5cc0c1ea0cd5ebf","txindex":1,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["mxWxxZF5pxTePC1pzvD8uDDRfnjZ7DX6UQ",100000],["mhcPGasbdmB8gfc3bZQ3ry8dvPzYN46A2P",663974]]}'

In [14]:
# function to create, sign, and send transaction 
def send_tx(coin, account, recipient, amount):
    
    raw_tx = create_tx(coin, account, recipient, amount)
    signed_tx = account.sign_transaction(raw_tx)
    
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [17]:
# testing send transaction function
send_tx(BTCTEST,btctest_account,btctest_address,0.001)